In [1]:
%load_ext autoreload
%autoreload 2

# Comparables

[Docs/Comparables](https://docs.urbandataanalytics.com/#tag/Comparables)


In [14]:
#!pip install folium pandas python-dotenv

## Search (Advanced)
[/api/integration/v2/comparables/search/advanced](https://docs.urbandataanalytics.com/#tag/Comparables/operation/api_integration_v2_comparables_search_advanced_get)

In [2]:
from utils import Search

search = Search().request(
    cadastral_ref='0990305VK4709B0007IS',
    no_cache=1,
    typology_type_id=1, # residentials
    property_type_id=4, # flats
    construction_type_id=2, # second-hands
    is_active=0, # unpublished
    distance=200,
    sort_by='distance',
    ascending=1,
)

cols = search.df.columns

print('\n', len(cols), 'features')
','.join(list(cols))

Total: 49 - Page: 1/2

 131 features


'_id,title,is_active,action_id,action_name,usage_id,usage_name,comparison_type,operation_type_id,operation_type_name,last_price,last_pm2,address,location,lat,lon,distance,similarity,property_type_id,property_type_name,subtypology_name,subtypology_type_id,typology_type_id,typology_type_name,construction_type_id,construction_type_name,build_status_id,build_status_name,date_in,date_out,first_date_origin,updated_at,date_diff,has_air_conditioner,has_common_areas,has_elevator,has_garage,has_garage_in_price,has_gym,has_pool,has_racket_court,has_security,has_storage,furnished,is_exterior,is_agency,is_bank,is_coop,is_short_rent,is_vpo,is_comparable,is_duplicated,cat_area,cat_rooms,category_id,n_rooms,n_baths,area,has_terrace,construction_year,floor,is_accurate_address,admin0,admin1,admin2,admin3,admin4,admin5,admin6,id_admin0,id_admin1,id_admin2,id_admin3,id_admin4,id_admin5,id_admin6,postal_code,geohash7,first_price,first_pm2,last_price_pm2_with_correction,last_price_with_correction,contact_ur

In [12]:
search.map()

In [35]:
search.df[:1].to_dict()

{'_id': {0: '54-103098737'},
 'title': {0: 'Piso en venta en Cuatro Caminos'},
 'is_active': {0: 0},
 'action_id': {0: 4},
 'action_name': {0: 'Unpublished w/ confirmed date'},
 'usage_id': {0: 1},
 'usage_name': {0: 'Residential'},
 'comparison_type': {0: 0},
 'operation_type_id': {0: 1},
 'operation_type_name': {0: 'Venta'},
 'last_price': {0: 332000.0},
 'last_pm2': {0: 4742.86},
 'address': {0: 'Barrio Cuatro Caminos'},
 'location': {0: {'coordinates': [-3.6976747, 40.4584394], 'type': 'Point'}},
 'distance': {0: 9},
 'similarity': {0: 2},
 'property_type_id': {0: 4},
 'property_type_name': {0: 'Apartment'},
 'subtypology_name': {0: 'Colective housing'},
 'subtypology_type_id': {0: 2},
 'typology_type_id': {0: 1},
 'typology_type_name': {0: 'Residencial'},
 'construction_type_id': {0: 2},
 'construction_type_name': {0: 'Segunda mano'},
 'build_status_id': {0: 1},
 'build_status_name': {0: 'Buen estado'},
 'date_in': {0: '2023-11-07'},
 'date_out': {0: '2023-11-28'},
 'first_date_or

In [3]:
from utils import show_images

show_images(search.df.loc[:3])

,_id,title,url,img1,img2,img3
0,66-1176540468,Piso en venta en Poeta Joan Maragall Cuatro Caminos - Azca,https://www.fotocasa . 76540468/d,,,
1,66-1181361150,"Piso en venta en Calle del Poeta Joan Maragall, 3, Cuatro Caminos - Azca",https://www.fotocasa . 1361150/d/,,,
2,54-103568328,Piso en venta en Cuatro Caminos,https://www.idealist . 103568328/,,,
3,54-103458638,Piso en venta en calle de Pedro Teixeira,https://www.idealist . 103458638/,,,


### Running more complex queries

In [7]:
params = {

    'address': 'Av de Brasil 3, Madrid',     
    'area': '100,200',
    'is_active': 1,
    'operation_type_id': 1,
    'distance': 1000,

    # in order to get a finer scoring and sort by similarity
    'target_area': 150,
    'target_floor': 3,
    'target_baths': 2,
    'target_rooms': 3,
    'target_property_type': 1, # penthouses
    'target_has_elevator': 1,
    'target_has_storage': 1,
    'target_is_exterior': 1,
    'target_has_terrace': 1,
    'target_build_status': 1,
    'sort_by': 'similarity',
    'ascending': 0,
}

search = Search().request(**params)

search.df[:3]

Total: 149 - Page: 1/5


,_id,title,is_active,action_id,action_name,usage_id,usage_name,comparison_type,operation_type_id,operation_type_name,...,score_build_status_id,score_has_elevator,score_has_storage,score_has_terrace,score_has_air_conditioner,score_has_garage,score_has_pool,score_is_exterior,score_property_type_id,score_distance
0,54-105013400,Ático en venta en calle de Marceliano Santa María,1,2,Old,1,Residential,1,1,Venta,...,100,100,100,100,None,None,None,100,100.0,88.947216
1,54-104894877,"Ático en venta en calle de Orense, 29",1,2,Old,1,Residential,1,1,Venta,...,100,100,0,100,None,None,None,100,100.0,97.597347
2,54-104594182,Ático en venta en Cuatro Caminos,1,2,Old,1,Residential,1,1,Venta,...,100,100,0,100,None,None,None,100,100.0,100.000000


In [8]:
show_images(search.df[:3])

,_id,title,url,img1,img2,img3
0,54-105013400,Ático en venta en calle de Marceliano Santa María,https://www.idealist . 105013400/,,,
1,54-104894877,"Ático en venta en calle de Orense, 29",https://www.idealist . 104894877/,,,
2,54-104594182,Ático en venta en Cuatro Caminos,https://www.idealist . 104594182/,,,


### Similarity scores

In [5]:
search = Search().request(
    cadastral_ref='0990305VK4709B0007IS',
    no_cache=1,
    typology_type_id=1, # residentials
    property_type_id=4, # flats
    construction_type_id=2, # second-hands
    is_active=0, # unpublished
    
    target_area = 120,
    target_property_type = 1, # penthouses
    target_build_status = 1,
    target_has_elevator = 1,
    target_has_storage = 1,
    target_is_exterior = 1,
    target_baths = 2,
    target_rooms = 3,
    
    distance=200,
    sort_by='distance',
    ascending=1,
)

search.df.describe().score

Total: 41 - Page: 1/2


count    30.000000
mean     76.938944
std      11.768105
min      57.673267
25%      66.089109
50%      79.331683
75%      87.252475
max      94.554455
Name: score, dtype: float64

In [6]:
search.score()

,_id,score,score_area,score_n_rooms,score_n_baths,score_floor,score_build_status_id,score_has_elevator,score_has_storage,score_has_terrace,score_has_air_conditioner,score_has_garage,score_has_pool,score_is_exterior,score_property_type_id,score_distance,similarity,distance
0,66-1178329933,62.623762,75.000000,50.000000,100.000000,25.000000,0,100,0,None,None,None,None,0,25.000000,100,1,44
1,54-103078289,85.396040,75.000000,100.000000,50.000000,25.000000,100,100,0,None,None,None,None,100,25.000000,100,2,45
2,54-100723433,70.792079,100.000000,100.000000,50.000000,100.000000,0,100,0,None,None,None,None,100,25.000000,100,1,49
3,54-104072229,79.950495,75.000000,75.000000,50.000000,75.000000,100,0,0,None,None,None,None,100,25.000000,100,1,76
4,54-97887545,70.792079,100.000000,100.000000,50.000000,100.000000,0,100,0,None,None,None,None,100,25.000000,100,1,81
5,66-1176254240,67.574257,100.000000,100.000000,100.000000,25.000000,0,100,0,None,None,None,None,0,25.000000,100,1,82
6,73-41677848696_102100,87.376238,100.000000,100.000000,100.000000,25.000000,100,100,0,None,None,None,None,0,25.000000,100,2,90
7,66-1165345109,57.673267,75.000000,75.000000,100.000000,25.000000,0,0,0,None,None,None,None,0,25.000000,100,0,93
8,73-28396504268_100200,78.712871,75.000000,100.000000,50.000000,100.000000,100,0,0,None,None,None,None,0,25.000000,100,1,100
9,66-1177370967,64.603960,75.000000,100.000000,100.000000,25.000000,0,100,0,None,None,None,None,0,25.000000,100,1,105


### uDA's Valuation (AVM)

In [12]:
search.df.valuation_sale.describe().round()

count        30.0
mean     415145.0
std       61127.0
min      327500.0
25%      369698.0
50%      405970.0
75%      449602.0
max      584810.0
Name: valuation_sale, dtype: float64

### Other interesting indicators

In [4]:
search = Search().request(
    address="Villaverde, Madrid", 
    typogy_type_id = 1,
    construction_type_id = 2,
    price='0,100000',
    area='50,150',
    operation_type_id=1, # sale
    gross_yield_asking_price = '5,12',
    target_area=100,
    is_active=1, # published
    sort_by='gross_yield_asking_price', 
    ascending=0,
    distance=1000,
)

maindrivers_zone = [c for c in search.df.columns if 'md_' in c]

cols = ['_id', 'title', 'admin3', 'admin4', 'admin5', 'url', 'last_price_with_correction'] 

cols += maindrivers_zone 
cols += ['time_on_market_sale','time_on_market_rent', 'output_absorption_rent', 'output_absorption_sale', 'price_gap', 'gross_yield_asking_price']

search.df[cols]

Total: 14 - Page: 1/1


,_id,title,admin3,admin4,admin5,url,last_price_with_correction,md_cycle_sale,md_liquidity_rent,md_liquidity_sale,md_volatility_sale,time_on_market_sale,time_on_market_rent,output_absorption_rent,output_absorption_sale,price_gap,gross_yield_asking_price
0,54-104922893,Local en venta en calle Alberto Palacios,Madrid,Villaverde,San Andrés,https://www.idealista.com/inmueble/104922893/,79000.00,Rising,AAA,BB,CCC,15,5,40.0,26.0,1.80,11.83
1,54-105064118,Piso en venta en calle Calamina,Madrid,Villaverde,San Andrés,https://www.idealista.com/inmueble/105064118/,77710.08,Rising,AAA,AA,CCC,9,3,61.0,49.0,44.73,11.52
2,54-102609410,Piso en venta en Villaverde Alto,Madrid,Villaverde,San Andrés,https://www.idealista.com/inmueble/102609410/,73382.40,Rising,AAA,AA,CCC,9,3,61.0,49.0,36.11,11.18
3,54-104644077,Piso en venta en paseo Talleres,Madrid,Villaverde,San Andrés,https://www.idealista.com/inmueble/104644077/,80192.00,Rising,AAA,AA,CCC,9,3,61.0,49.0,52.61,11.01
4,54-104885544,Piso en venta en calle de Albino Hernández Lázaro,Madrid,Villaverde,San Andrés,https://www.idealista.com/inmueble/104885544/,77056.00,Rising,AAA,AA,CCC,9,3,61.0,49.0,37.54,10.57
5,54-102978094,Piso en venta en Los Ángeles,Madrid,Villaverde,San Andrés,https://www.idealista.com/inmueble/102978094/,76070.40,Rising,AAA,AA,CCC,9,3,61.0,49.0,15.45,10.25
6,54-104632705,Local en venta en Villaverde Alto,Madrid,Villaverde,San Andrés,https://www.idealista.com/inmueble/104632705/,97500.00,Rising,AAA,BB,CCC,15,5,40.0,26.0,-13.38,9.78
7,54-104146404,"Local en venta en calle Lenguas, 29 -21",Madrid,Villaverde,San Andrés,https://www.idealista.com/inmueble/104146404/,92000.00,Rising,AAA,BB,CCC,15,5,40.0,26.0,-13.75,9.67
8,54-104833627,Local en venta en calle de Albino Hernández Lá...,Madrid,Villaverde,San Andrés,https://www.idealista.com/inmueble/104833627/,100000.00,Rising,AAA,BB,CCC,15,5,40.0,26.0,-24.05,9.31
9,54-105121562,Piso en venta en Villaverde Alto,Madrid,Villaverde,San Andrés,https://www.idealista.com/inmueble/105121562/,87808.00,Rising,AAA,AA,CCC,9,3,61.0,49.0,1.42,8.57


### Cadastral matchs

In [13]:
search = Search().request(
    address="Calle de Huesca 27, Madrid", 
    operation_type_id=1, # sale
    is_active=1, # published
    sort_by='similarity', 
    ascending=0
)

def read_match(x):
    print(f'\n{x._id} - {x.title} {x.url}')
    if match:=x.cadastral_match:
        match = sorted(match, key = lambda x: -x['score'])
        for c in match[:3]:
            print('\t', c.get('address'), c.get('floor'), c.get('cadastral_bi'), c.get('area_total_con_elementos_comunes'), c.get('score_10'))
        print('\t...')
        return
    print('\t...')
        
_ = search.df[:5].apply(lambda x: read_match(x), axis=1)

Total: 118 - Page: 1/4

54-104965509 - Piso en venta en calle de Huesca, 19 https://www.idealista.com/inmueble/104965509/
	 CALLE HUESCA 19, MADRID (MADRID) 2 0890408VK4709B0008WD 85.0 7.0
	 CALLE HUESCA 19, MADRID (MADRID) 2 0890408VK4709B0009EF 79.0 7.0
	 CALLE HUESCA 19, MADRID (MADRID) 2 0890408VK4709B0007QS 71.0 7.0
	...

54-104869914 - Piso en venta en Cuzco-Castillejos https://www.idealista.com/inmueble/104869914/
	 CALLE ROBLE 1, MADRID (MADRID) 6 0994803VK4709D0029MM 146.0 0.0
	 CALLE SAN GERMAN 57, MADRID (MADRID) 6 0789101VK4708H0214MD 157.0 0.0
	 CALLE ROBLE 3, MADRID (MADRID) 6 0994803VK4709D0060LL 150.0 0.0
	...

66-1182650959 - Piso en venta en Calle Hermano Gárate, Castillejos - Cuzco https://www.fotocasa.es/es/comprar/vivienda/madrid-capital/calefaccion-ascensor/182650959/d
	...

54-104072229 - Piso en venta en calle de San Germán https://www.idealista.com/inmueble/104072229/
	 CALLE SAN GERMAN 37, MADRID (MADRID) 2 0888210VK4708H0026WH 109.0 4.0
	 CALLE SAN GERMAN 37,

### Closings matchs

In [14]:
search = Search().request(
    address="calle huescsa 27, Madrid", 
    operation_type_id=1, # sale
    is_active=0, # published
    target_area=100,
    sort_by='score', 
    ascending=0
)

def read_match(x):
    print(f'\n{x._id} - {x.title} {x.score}')
    if match:=x.closing_match:
        match = sorted(match, key = lambda x: -x['score'])
        for c in match[:3]:
            print('\t', 'id:', c.get('clo_id_uda'), 'score:', c.get('score'))
        print('\t...')
        return
    print('\t...')
        
_ = search.df[:10].apply(lambda x: read_match(x), axis=1)

Total: 250 - Page: 1/9

66-1178501558 - Piso en venta en Castillejos - Cuzco 100.0
	...

54-103078289 - Piso en venta en calle de la Infanta Mercedes, 34 100.0
	 id: 41-5cfc122de6a582b7c22de712570c6c88_1 score: 5
	 id: 41-d0b63cceaf50d7644ea32e6ee1d63f95_1 score: 5
	 id: 41-6b9fc394b993ef676789e664536e3851_1 score: 5
	...

54-102935025 - Piso en venta en calle Hermano Gárate 100.0
	...

54-100804924 - Piso en venta en calle Hermano Gárate, 1 100.0
	 id: 41-711fe67fa356eabd2abbda81f622e25d_1 score: 0
	 id: 41-9719b5e72597353a7b76e007675e32cc_1 score: 0
	 id: 41-4b88862b53c6a236c2a27abca2260b53_1 score: 0
	...

54-101584916 - Dúplex en venta en Alonso Castrillo, 19 100.0
	 id: 41-c26b6334135ac16f497c05860868219f_1 score: 0
	 id: 41-aa2d1d559b49d62c73d8adb2617938a5_1 score: 0
	 id: 41-0267ac6c82dd86428654aec714862a56_1 score: 0
	...

54-102492870 - Piso en venta en calle de la Infanta Mercedes 100.0
	...

66-1165345109 - Piso en venta en Calle del Marqués de Leis 16 Castillejos - Cuzco 10

### Listing one or multiple records

In [3]:
# Read the file content
import pandas as pd 
from utils import Search
import io

ref_list = []

df = pd.read_csv('selection.csv')

if 'cadastral_ref' in df.columns:
    ref_list = df['cadastral_ref'].tolist()

search = None

for ref in ref_list[:1]: 
    
    search = Search().request(
        
        cadastral_ref=ref,
        distance=2000,
        # typology is set automaticaly
        # property_type_id is set automaticaly
        # target_area is set automaticaly as +- 20% of asset' area
        # target floor is set from cadastral as well
    )
    
    search.df.to_csv(f'{ref}-comparables.csv')

search.map()

Total: 12 - Page: 1/1
